In [3]:
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.stats import ttest_ind,ttest_rel
from scipy import stats,signal
import matplotlib.pyplot as plt

import seaborn as sns

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import platform
import os
import pandas as pd

import imageio

# add the top level directory to allow it to be imported
import sys
sys.path.append(os.path.split(os.getcwd())[0])
from utilities import plotting_utilities
from utilities import helper_functions

from visual_behavior_research.projects.tbd.session import Session

import visual_behavior_research.projects.tbd.utilities as tbdu
from visual_behavior_research.projects.tbd.Inscopix.series import SeriesCellSet
import visual_behavior_research.plotting as vbp
import visual_behavior_research.plotting.dro_plots as pf
import visual_behavior_research.utilities as vbu

from visual_behavior_research.projects.tbd.identify_repeat_rois import Repeat_ROI_Identifier

import isx
from visual_behavior_research.projects.tbd.Inscopix.pipeline import make_cell_map,plot_cell_map

from visual_behavior.translator.foraging import data_to_change_detection_core
from visual_behavior.translator.core import create_extended_dataframe

pd.set_option('display.max_columns', 500)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
data_summary = tbdu.get_data_inventory()
data_summary.head()

,id,basepath,behavior_stage,folder,genotype,good_behavior,good_for_analysis,good_neural_data,good_sync,manually_checked,mouse_id,notes,problem,problem_description,processed,running_data,verified_files_present,size(gb),experiment_type,single_cell_plots_made,single_cell_images_made,number_accepted_cells,full_path,recording_date,processing_date,summarized
0,0,//olsenlab1/data/endoscope_imaging,autoreward,2018.03.07_M332849_DoC_day1_autoreward,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,yes,yes,yes,yes,0,M332849,None,1,Not organized into 10 minute chunks,0.0,yes,None,74.790761,change_detection,None,None,NaN,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-03-07 15:16:45.876000,2018-12-15 15:53:28.984200001-08:00,None
1,1,//olsenlab1/data/endoscope_imaging,static_grating,2018.03.09_M332849_DoC_day2,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,no,no,yes,yes,1,M332849,None,0,None,1.0,yes,None,151.789864,change_detection,True,True,45.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-03-09 11:27:50.579000,2018-07-25 12:41:22.762362242-07:00,True
2,2,//olsenlab1/data/endoscope_imaging,static_grating,2018.03.13_M332849_DoC_day4,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,yes,yes,yes,yes,1,M332849,None,0,None,1.0,yes,None,134.175156,change_detection,True,True,64.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-03-13 11:26:45.003000,2018-07-20 10:57:53.647888899-07:00,True
3,3,//olsenlab1/data/endoscope_imaging,natural_images,2018.03.30_M332849_DoC_day9,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,yes,yes,yes,yes,1,M332849,None,0,Note: more dead cells in forcus this day,1.0,yes,None,146.164498,change_detection,True,True,57.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-03-30 13:53:41.511000,2018-07-25 13:48:01.009045124-07:00,True
4,4,//olsenlab1/data/endoscope_imaging,natural_images,2018.04.3_M332849_DoC_day11,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,no,no,yes,no,1,M332849,None,0,None,1.0,yes,None,144.396109,change_detection,True,True,108.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-04-03 13:20:29.101000,2018-07-25 13:59:23.514958143-07:00,True


In [5]:
last_sessions = pd.DataFrame(data_summary.query('good_for_analysis == "yes"').groupby('mouse_id')['recording_date'].max()).reset_index() # last session for every mouse
sessions_to_analyze = last_sessions.merge(
    data_summary,
    left_on=['mouse_id','recording_date'],
    right_on=['mouse_id','recording_date'],
)
sessions_to_analyze

,mouse_id,recording_date,id,basepath,behavior_stage,folder,genotype,good_behavior,good_for_analysis,good_neural_data,good_sync,manually_checked,notes,problem,problem_description,processed,running_data,verified_files_present,size(gb),experiment_type,single_cell_plots_made,single_cell_images_made,number_accepted_cells,full_path,processing_date,summarized
0,M332849,2018-06-14 12:44:33.658000,12,//olsenlab1/data/endoscope_imaging,natural_images,2018.06.14_M332849_DoC_day23,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,yes,yes,yes,no,1,None,0,None,1.0,yes,None,146.340312,change_detection,True,True,126.0,//olsenlab1/data/endoscope_imaging/M332849/201...,2018-07-26 12:11:01.983524799-07:00,True
1,M348126,2018-06-13 13:47:38.463000,29,//olsenlab1/data/endoscope_imaging,natural_images,2018.06.13_M348126_DoC_Day20,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai94(TITL-...,yes,yes,yes,no,1,None,0,None,1.0,yes,None,147.288604,change_detection,True,True,103.0,//olsenlab1/data/endoscope_imaging/M348126/201...,2018-07-26 18:15:45.828682184-07:00,True
2,M351181,2018-03-14 14:32:15.328000,37,//olsenlab1/data/endoscope_imaging,natural_images,2018.03.14__M351181_DoC_day8,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai94(TITL-...,yes,yes,yes,yes,1,None,0,None,1.0,yes,None,147.362931,change_detection,True,True,127.0,//olsenlab1/data/endoscope_imaging/M351181/201...,2018-07-26 18:59:36.293976068-07:00,True
3,M397408,2018-11-01 11:40:14.487000,153,//olsenlab1/data/endoscope_imaging,None,20181101_M397408_DoC_day5,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,yes,None,None,0,,0,,1.0,None,None,153.154207,change_detection,True,True,29.0,//olsenlab1/data/endoscope_imaging/M397408/201...,2019-01-22 01:32:10.700291872-08:00,True
4,M423780,2019-06-11 10:38:31,245,//olsenlab1/data/endoscope_imaging,None,2019.06.11_DoC,Gnb4-IRES2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-...,None,yes,None,None,None,None,0,,1.0,None,None,183.495635,change_detection,True,True,61.0,//olsenlab1/data/endoscope_imaging/M423780/201...,2019-06-26 23:56:50.550086260-07:00,True


In [6]:
data = {}

folders = sessions_to_analyze['folder']

for foldername in folders:
    print('on folder {}'.format(foldername))

    row = data_summary[data_summary['folder']==foldername].iloc[0]

    mouse_id = row['mouse_id']
    path = os.path.join(row['basepath'],row['mouse_id'],row['folder'])

    data[foldername]=pd.read_pickle(os.path.join(path,'cached_data_2019.10.04.pkl'))

    data[foldername]['session'] = Session(
        data[foldername]['path'], 
        load_cached_traces_table = True,
        behavior_pkl_filename = fig3_helper_functions.which_pkl(data[foldername]['filename_dict'])
    )

on folder 2018.06.14_M332849_DoC_day23


NameError: name 'fig3_helper_functions' is not defined